# Issue Classification Example

In this example we perform a logical regression on a dataset of github issues to predict the labels on newly entered tickets.

In a second step we try to find issues that deal with similar problems.

In [ ]:
# required libraries
from github import Github
import os
import pandas as pd
import numpy as np
import pickle

In [ ]:

# Data Preparation 

## Retrieve the issues from github

access_token = USER = os.getenv('GH_API_ACCESS')
token = Github(access_token)
repo = token.get_repo('quarkusio/quarkus')

# force_fetch = False
# if (force_fetch or not os.path.exists('../data/issues.pkl')): 
#     issues = repo.get_issues(state='open')  
#     pickle.dump(issues, open('../data/issues.pkl', 'wb'))
# else:
#     print("Loading issues from file...")
#     issues = pickle.load(open("../data/issues.pkl", 'rb'))


issues = repo.get_issues(state='open')  
print("Number of issues in total: ", issues.totalCount)

cols = columns = ['number', 'title', 'body', 'labels', 'state']
df = pd.DataFrame(columns = cols)
unique_labels = set()

for issue in issues.get_page(0):
    lable_names = []
    for label in issue.get_labels():
        lable_names.append(label.name)
        unique_labels.add(label.name)
    new_record = pd.DataFrame([[issue.number, issue.title, issue.body, lable_names, issue.state]], columns=cols)
    df = pd.concat([df, new_record], ignore_index=True)    

print("Unique labels ({0}): {1}".format(len(unique_labels), unique_labels))
df.head()